### OSC-CO2: Coattention and Cosegmentation Framework For Object State Change With Multiple Features 

#### Code by: Rubi Quiñones

If you use any part of this code or methodology, please cite our paper: 

R. Quiñones, F. Munoz-Arriola, S. Das Choudhury, A. Samal. "CoPPNet: A Cosegmentation Framework for High-Throughput Binary Plant Image Sequences." IEEE, (2022).

## Import Packages

In [1]:
import torch
import torchvision
import matplotlib.pyplot as plt
import numpy as np
import tqdm.notebook
from torch.utils.tensorboard import SummaryWriter
import datetime
from models import ResNet50, FCN32s
import tensorflow as tf
from PIL import Image
import statistics
from itertools import tee, islice, chain
import os
from os import walk
import math

## User-Defined Variables

In [23]:
num_prior_algo = 2 # this is equal to batch
dataset_path = './CosegPP_extension/Sunflower-D-1'
nb_epochs = 7

class_list = [ item for item in os.listdir(dataset_path) if os.path.isdir(os.path.join(dataset_path, item)) ]
print (class_list)
class_list_len = len(class_list)

['Vis_SV_72', 'Fluo_SV_144', 'Vis_SV_144', 'Vis_SV_0', 'Fluo_SV_216', 'Fluo_SV_72', 'IR_SV_0', 'Vis_SV_216', 'Fluo_SV_0', 'IR_SV_144', 'IR_SV_216', 'IR_SV_72']


## Defining Functions

In [24]:
def select_class(dataset, class_name):
    class_idx = dataset.class_to_idx[class_name]
    dataset = [element for element in dataset if element[1]==class_idx]
    return dataset
       
def imshow(img):
    npimg = img.numpy()
    plt.imshow((np.transpose(npimg, (1, 2, 0)) * 255).astype(np.uint8))
    
def save_image_mask_segmented(data_loader, timestamp_list):
    timestamp_counter = 0
    dataiter = iter(data_loader)
    for i in range(math.ceil(len(timestamp_list)/num_prior_algo)): 
        images, labels = next(dataiter)

        with torch.no_grad():
            masks = generator(images)

        for j in range(num_prior_algo):
            try: 
                path = './output/images/' + dataset_path[2:] + '/' + class_item + '/' + timestamp_list[timestamp_counter]
                plt.figure()
                images[j, 0,:] = images[j, 0,:] * 0.229 + 0.485
                images[j, 1,:] = images[j, 1,:] * 0.224 + 0.456
                images[j, 2,:] = images[j, 2,:] * 0.225 + 0.406  
                imshow(images[j])
                plt.axis('off')
                plt.savefig(path,bbox_inches='tight',pad_inches = 0)
                plt.close()

                path = './output/masks/' + dataset_path[2:] + '/' + class_item + '/' + timestamp_list[timestamp_counter]
                plt.figure()
                imshow(masks[j])
                plt.axis('off')
                plt.savefig(path,bbox_inches='tight',pad_inches = 0)
                plt.close()

                path = './output/segmented/' + dataset_path[2:] + '/' + class_item + '/' + timestamp_list[timestamp_counter]
                plt.figure()
                segmented = images[j] * masks[j]
                imshow(segmented)
                plt.axis('off')
                plt.savefig(path,bbox_inches='tight',pad_inches = 0) 
                plt.close()
            except:
                continue
            timestamp_counter = timestamp_counter + 1

def previous_and_next(some_iterable):
    prevs, items, nexts = tee(some_iterable, 3)
    prevs = chain([None], prevs)
    nexts = chain(islice(nexts, 1, None), [None])
    return zip(prevs, items, nexts)

## Initialize Extractor and Generator

In [25]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

transform = torchvision.transforms.Compose([
                            torchvision.transforms.Resize((384,384)),
                            torchvision.transforms.ToTensor(),
                            torchvision.transforms.Normalize([0.485, 0.456, 0.406],[0.229, 0.224, 0.225])])

feature_extractor =  ResNet50().eval()
generator = FCN32s().train()

fake_feature_extractor = ResNet50().eval()
fake_generator = FCN32s().train()

optimizer = torch.optim.SGD(generator.parameters(), lr = 1e-2)
criterion = torch.nn.MSELoss()
feature_extractor.to(device)
generator.to(device)
fake_feature_extractor.to(device)
fake_generator.to(device)

FCN32s(
  (pretrained_net): VGGNet(
    (features): Sequential(
      (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): ReLU(inplace=True)
      (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (3): ReLU(inplace=True)
      (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (6): ReLU(inplace=True)
      (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (8): ReLU(inplace=True)
      (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (11): ReLU(inplace=True)
      (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (13): ReLU(inplace=True)
      (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (15): ReLU(inplace=True)
 

## Stage 1: Object Mask Refinement

In [26]:
for i, class_item in enumerate(class_list) : 
    
    # resetting for each class
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    transform = torchvision.transforms.Compose([
                                torchvision.transforms.Resize((384,384)),
                                torchvision.transforms.ToTensor(),
                                torchvision.transforms.Normalize([0.485, 0.456, 0.406],[0.229, 0.224, 0.225])])
    feature_extractor =  ResNet50().eval()
    generator = FCN32s().train()
    fake_feature_extractor = ResNet50().eval()
    fake_generator = FCN32s().train()
    optimizer = torch.optim.SGD(generator.parameters(), lr = 1e-2)
    criterion = torch.nn.MSELoss()
    feature_extractor.to(device)
    generator.to(device)
    fake_feature_extractor.to(device)
    fake_generator.to(device)
    
    # creating class dataset
    dataset = torchvision.datasets.ImageFolder(dataset_path, transform=transform)
    class_dataset = select_class(dataset, class_item)
    data_loader = torch.utils.data.DataLoader(class_dataset, shuffle = False, batch_size= num_prior_algo) # data_loader = ceil((no. of samples in dataset)/batchsize)
   
    dataset_class_path = dataset_path + '/' + class_item
    timestamp_list = next(walk(dataset_class_path), (None, None, []))[2]
    timestamp_list.sort()
    
    if (timestamp_list[0] == '.DS_Store'):
        timestamp_list.pop(0)

    print ("In: ", dataset_path)
    print ("Starting: ", class_item)
    
    ##########################
    #### Starting CoPPNet ####
    ##########################

    now = datetime.datetime.now()
    writer = SummaryWriter('./logs/{}:{}:{}'.format(now.hour, now.minute, now.second))
    #torch.autograd.set_detect_anomaly(True) # for debugging 

    for epoch in range(nb_epochs):

        # initialize running losses 
        running_loss_coattention = 0
        running_loss_temporal = 0 
        running_loss_coseg = 0 

        pbar = tqdm.notebook.tqdm(total=len(data_loader))    

        for prev_imgs, images, next_imgs in previous_and_next(data_loader):

            optimizer.zero_grad()

            # initialize losses
            proposals = [None] * len(data_loader) 
            loss_coattention = 0
            loss_temporal = 0 
            loss_coseg = 0 

            #########################################################
            #### procesing current and prior images using        ####
            #### FCN (generator) and ResNet50(feature_extractor) ####
            #########################################################

            # process current images to get masks and features  
            # where segmented_objects = masks * images
            # where segmented_backgrounds = (1 - masks) * images
            images = images[0].to(device)        
            masks = generator(images) 
            object_features = feature_extractor(torch.mul(images, masks)).squeeze()
            background_features = feature_extractor(torch.mul(images, (1 - masks))).squeeze()

            # process previous images to get masks and features 
            if prev_imgs is not None :
                prev_imgs = prev_imgs[0].clone().to(device)  
                prev_masks = fake_generator(prev_imgs)
                prev_segmented_objects = prev_masks * prev_imgs
                prev_object_features = fake_feature_extractor(prev_segmented_objects).squeeze()

            ###################################################
            #### starting loss computations based on epoch ####
            ###################################################

            if (epoch < math.floor(nb_epochs/3)): 

                loss_coattention = masks.mean() * 2
                loss_temporal = masks.mean() * 2
                loss_coseg = masks.mean() * 2
            
            elif ((epoch >= math.floor(nb_epochs/3)) & (epoch <= (nb_epochs-2))):
                
                for q in range(images.shape[0]):
                    for j in range(images.shape[0]):
                        if q == j:
                            continue
                        #### computing coattention loss from Hsu18 ####
                        batch, dim = object_features.size()
                        dij_plus = (torch.dist(object_features[q,:], object_features[j,:], p=2) ** 2) / dim / batch
                        dij_minus = (torch.dist(object_features[q,:], background_features[q,:], p=2) ** 2) / dim / batch
                        dij_minus = dij_minus + (torch.dist(object_features[j,:], background_features[j,:], p=2) ** 2) / dim / batch
                        dij_minus = dij_minus/2
                        loss_coattention = loss_coattention + torch.log(torch.exp(-dij_plus)/(torch.exp(-dij_plus) + torch.exp(-dij_minus))) * (-1)
                
                loss_temporal = masks.mean() * 2
                loss_coseg = masks.mean() * 2
                  
            else:
                if prev_imgs is not None :
                    #### computing cosegmentation loss ####
                    obj_A = feature_extractor(torch.mul(images, masks)).squeeze() # object_features
                    back_A = feature_extractor(torch.mul(images, 1.0 - masks)).squeeze() # background features 
                    obj_B = feature_extractor(torch.mul(prev_imgs, prev_masks)).squeeze() # prev_object_features
                    back_B = feature_extractor(torch.mul(prev_imgs, 1.0 - prev_masks)).squeeze() # prev_background features
                    pos = criterion(obj_A, obj_B) ** 2
                    neg = ((criterion(obj_A, back_B) ** 2) + (criterion(obj_B, back_B) ** 2))/2
                    neg = torch.max(torch.tensor(0), 2 - neg)
                    loss_coseg = loss_coseg + (pos + neg) 

                for q in range(images.shape[0]):
                    for j in range(images.shape[0]):
                        if q == j:
                            continue
                        #### computing coattention loss from Hsu18 ####
                        batch, dim = object_features.size()
                        dij_plus = (torch.dist(object_features[q,:], object_features[j,:], p=2) ** 2) / dim / batch
                        dij_minus = (torch.dist(object_features[q,:], background_features[q,:], p=2) ** 2) / dim / batch
                        dij_minus = dij_minus + (torch.dist(object_features[j,:], background_features[j,:], p=2) ** 2) / dim / batch
                        dij_minus = dij_minus/2
                        loss_coattention = loss_coattention + torch.log(torch.exp(-dij_plus)/(torch.exp(-dij_plus) + torch.exp(-dij_minus))) * (-1)
                        
                        #### computing temporal loss ####
                        ob_minus = criterion(prev_object_features[q,:].clone(), prev_object_features[j,:].clone()) ** 2
                        ob_plus = criterion(object_features[q,:], object_features[j,:]) ** 2
                        loss_temporal = loss_temporal + torch.log(torch.exp(-ob_plus.clone())/(torch.exp(-ob_plus.clone()) + torch.exp(-ob_minus))) * (-1)

            (1.0*loss_coattention + 0.5*loss_temporal + 0.5*loss_coseg).backward(retain_graph=True) 
            optimizer.step()
            
            with torch.no_grad():
                try :
                    running_loss_coattention += loss_coattention.item()
                except: # odd number of images so could not compute last image 
                    running_loss_coattention += 0
                try :
                    running_loss_temporal += loss_temporal.item()
                except: # odd number of images so could not compute last image 
                    running_loss_temporal += 0
                try : 
                    running_loss_coseg += loss_coseg 
                except:
                    running_loss_coseg += 0
            pbar.update(1)

        if epoch >= math.floor(nb_epochs/3):
            writer.add_scalar('training_loss',running_loss_coattention/len(dataset), epoch)
        print('Epoch {} coattention loss {}'.format(epoch, running_loss_coattention/len(dataset))) 
        print('Epoch {} temporal loss {}'.format(epoch, running_loss_temporal/len(dataset))) 
        print('Epoch {} coseg loss {}'.format(epoch, running_loss_coseg/len(dataset))) 

    save_image_mask_segmented(data_loader, timestamp_list)

In:  ./CosegPP_extension/Sunflower-D-1
Starting:  Vis_SV_72


  0%|          | 0/13 [00:00<?, ?it/s]

Epoch 0 coattention loss 0.028955393993273015
Epoch 0 temporal loss 0.028955393993273015
Epoch 0 coseg loss 0.02895539626479149


  0%|          | 0/13 [00:00<?, ?it/s]

Epoch 1 coattention loss 0.014848776888556597
Epoch 1 temporal loss 0.014848776888556597
Epoch 1 coseg loss 0.014848776161670685


...

Epoch 4 coattention loss 0.05657816369359086
Epoch 4 temporal loss 0.009544476003545086
Epoch 4 coseg loss 0.009544476866722107


  0%|          | 0/14 [00:00<?, ?it/s]

Epoch 5 coattention loss 0.05536060325983094
Epoch 5 temporal loss 0.009060902521014214
Epoch 5 coseg loss 0.009060902521014214


  0%|          | 0/14 [00:00<?, ?it/s]

Epoch 6 coattention loss 0.053903895543842784
Epoch 6 temporal loss 0.059089873258660476
Epoch 6 coseg loss 0.08035019785165787
